# Practica 1: Predicción de la producción de energía eólica con SCIKIT-LEARN
Autores: Rodrigo Maroto Caño y Rodrigo Toldos Frutos  
Correos electrónicos: 100454455@alumnos.uc3m.es - 100454470@alumnos.uc3m.es   
Usuarios en GitHub: @RodrigoMaroto (owner) y @100454470

## Análisis exploratorio de datos (EDA)

Cargamos el dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
wind_ava = pd.read_csv('dataset/wind_ava.csv.gz', compression="gzip")

Seleccionamos la columna "datetime" como indice tras convertirla a un objeto DateTime y descartamos todas las columnas que no sean "energy" (nuestra columna de predicción) y las variables de la localización 13 Sotavento.

In [3]:
wind_ava["datetime"] = pd.to_datetime(wind_ava["datetime"])
wind_ava.set_index("datetime", inplace=True)
for c in wind_ava.columns:
  if not c.endswith(".13") and c != "energy":
    wind_ava.drop(c, axis=1, inplace=True)


A continuación, mostramos los datos estadísticos de las variables restantes. Se puede observar que existen 4748 instancias y 22 características. Todas las variables son numéricas. Además podemos observar que todas las desviaciones típicas (std) son distintas de 0, por lo que ninguna columna es constante. El objetivo es predecir la cantidad de energía generada por lo que este es un problema de regresión.

In [4]:
wind_ava.describe()

,energy,p54.162.13,p55.162.13,cape.13,p59.162.13,lai_lv.13,lai_hv.13,u10n.13,v10n.13,sp.13,...,t2m.13,stl2.13,stl3.13,iews.13,inss.13,stl4.13,fsr.13,flsr.13,u100.13,v100.13
count,4748.000000,4.748000e+03,4748.000000,4748.000000,4.748000e+03,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,...,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000
mean,693.126247,2.489477e+06,16.008810,31.166541,1.706692e+06,2.815222,2.576284,0.386215,0.120528,97820.301287,...,285.689253,286.663838,286.665988,0.074229,0.049971,286.668152,0.413677,-5.908467,0.447175,0.328204
std,665.531609,4.482599e+04,6.552216,121.758977,1.466953e+06,0.397377,0.116434,3.100583,3.016766,713.689654,...,6.163483,5.547947,4.582827,0.367013,0.379014,3.552873,0.007602,0.094359,4.841730,4.667552
min,0.010000,2.358748e+06,1.650268,0.000000,5.610341e+04,2.323973,2.425866,-8.619823,-8.867441,93770.364813,...,268.970603,275.461648,278.389271,-1.714897,-1.438829,280.875389,0.364805,-6.130465,-11.879053,-13.043453
25%,144.170000,2.458543e+06,11.203264,0.000000,6.563209e+05,2.425944,2.461630,-1.950008,-2.050920,97459.369264,...,281.458939,282.287394,282.689506,-0.126880,-0.148495,283.405549,0.410027,-5.977599,-3.836853,-3.256194
50%,465.305000,2.490478e+06,15.543441,1.004148,1.239176e+06,2.758857,2.560520,0.048820,-0.191853,97861.147677,...,285.395453,286.191880,286.204914,0.010551,-0.003569,286.591659,0.410917,-5.947220,0.282399,-0.389416
75%,1089.375000,2.525134e+06,20.214077,14.143328,2.296548e+06,3.205385,2.688526,2.641779,2.016289,98251.478418,...,289.740438,291.345311,290.989045,0.232378,0.172325,290.024705,0.417070,-5.858848,4.187953,3.614395
max,2792.550000,2.580387e+06,39.230807,2311.662152,1.110694e+07,3.450745,2.762992,12.974802,11.699814,99917.733093,...,305.000640,299.556292,295.639998,2.842552,2.366522,292.808658,0.428914,-5.618172,18.964137,16.913033


Calculamos los missing values, tanto NULL como NaN, de cada columna y no existe ninguno de estos datos en todo el dataset. 

In [5]:
wind_ava.isna().sum()

energy        0
p54.162.13    0
p55.162.13    0
cape.13       0
p59.162.13    0
lai_lv.13     0
lai_hv.13     0
u10n.13       0
v10n.13       0
sp.13         0
stl1.13       0
u10.13        0
v10.13        0
t2m.13        0
stl2.13       0
stl3.13       0
iews.13       0
inss.13       0
stl4.13       0
fsr.13        0
flsr.13       0
u100.13       0
v100.13       0
dtype: int64

In [6]:
wind_ava.isnull().sum()

energy        0
p54.162.13    0
p55.162.13    0
cape.13       0
p59.162.13    0
lai_lv.13     0
lai_hv.13     0
u10n.13       0
v10n.13       0
sp.13         0
stl1.13       0
u10.13        0
v10.13        0
t2m.13        0
stl2.13       0
stl3.13       0
iews.13       0
inss.13       0
stl4.13       0
fsr.13        0
flsr.13       0
u100.13       0
v100.13       0
dtype: int64

## Selección de Modelo y Optimización de Hiperparámetros

### Metodología de evaluación

Nuestro dataset es una serie temporal por lo que la validación y medición del rendimiento debe tener esto en cuenta. Nuestra outer validation usará los datos del último año, 2009, como test set y para la inner usaremos TimeSeriesSplits para generar los 3 folds para hacer el ajuste de hiperparámetros con validación cruzada.   
Para la evaluación de los modelos usaremos la raíz de error cuadrático medio (RMSE), que lo haremos a traves de la función que ofrece scikit-learn llamada root_mean_squared_error. Además, de manera informativa también calcularemos el r2 para obtener otra medida de rendimiento agnóstica al problema.


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error, r2_score

In [8]:
# Outer Loop
test_size = len([0 for x in wind_ava.index if x.year >=2009])
train, test = train_test_split(wind_ava, test_size=test_size, shuffle=False)
X_train = train.drop(['energy'], axis='columns')
y_train = train[['energy']]
X_test = test.drop(['energy'], axis='columns')
y_test = test[['energy']]
#Inner Loop
tss = TimeSeriesSplit(n_splits=3)

In [9]:
for k, data in enumerate(tss.split(X_train, y_train)):
  cv_train, cv_val = data
  print(f"En la iteración {k+1}, los índices son:")
  print(f"  Train: {cv_train[0]} - {cv_train[-1]}")
  print(f"  Val: {cv_val[0]} - {cv_val[-1]}")

En la iteración 1, los índices son:
  Train: 0 - 958
  Val: 959 - 1914
En la iteración 2, los índices son:
  Train: 0 - 1914
  Val: 1915 - 2870
En la iteración 3, los índices son:
  Train: 0 - 2870
  Val: 2871 - 3826


### Selección de Scaler

En esta sección elegiremos el mejor scaler usando KNN haciendo validación cruzada con nuestro conjunto de entrenamiento y una pequeña optimización de hiperparámetros (HPO). 

In [10]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV

scaler_scores = {}
param_grid = {"knn__n_neighbors":[3,5,7,9,11,13,15]}
param_grid_b = {"n_neighbors":[3,5,7,9,11,13,15]}

In [11]:
# Standard Scaler
pipeline_std = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsRegressor())
])
grid_std = GridSearchCV(pipeline_std, param_grid, cv=tss, scoring="neg_root_mean_squared_error")
grid_std.fit(X_train, y_train)
scaler_scores["StandardScaler"] = -grid_std.best_score_

In [12]:
# MinMax Scaler
pipeline_minmax = Pipeline([
    ("scaler", MinMaxScaler()),
    ("knn", KNeighborsRegressor())
])
grid_minmax = GridSearchCV(pipeline_minmax, param_grid, cv=tss, scoring="neg_root_mean_squared_error")
grid_minmax.fit(X_train, y_train)
scaler_scores["MinMaxScaler"] = -grid_minmax.best_score_

In [13]:
# Robust Scaler
pipeline_robust = Pipeline([
    ("scaler", RobustScaler()),
    ("knn", KNeighborsRegressor())
])
grid_robust = GridSearchCV(pipeline_robust, param_grid, cv=tss, scoring="neg_root_mean_squared_error")
grid_robust.fit(X_train, y_train)
scaler_scores["RobustScaler"] = -grid_robust.best_score_

In [14]:
# No scaler
grid_knn = GridSearchCV(KNeighborsRegressor(), param_grid_b, cv=tss, scoring="neg_root_mean_squared_error")
grid_knn.fit(X_train, y_train)
scaler_scores["KNN w/o scaler"] = -grid_knn.best_score_

In [15]:
for name, score in scaler_scores.items():
    print(f"{name}: {score}")

StandardScaler: 434.8935360208871
MinMaxScaler: 472.164886672966
RobustScaler: 452.59145275866126
KNN w/o scaler: 609.373666055656


En estos resultados podemos observar en primer lugar, la diferencia notable que produce el escalado de datos en modelos de KNN. Además para nuestro dataset el método de escalado que mejor rendimiento ofrece es el Standard, que realiza la conocida normalización de la distribución normal que consiste en restar la media y dividir por la varianza del conjunto de entrenamiento. Este escalador es el que usaremos de aqui en adelante para los modelos que se beneficien de ello como KNN o SVM.

### Regresor dummy

En esta sección podremos ver las métricas (RMSE y R²) asociadas a un modelo de regresión trivial (dummy regressor), para posteriormente poder comparar los resultados de nuestros modelos con los de este dummy regressor y ver si realmente son buenos o no.

In [16]:
from sklearn.dummy import DummyRegressor
dummy = DummyRegressor().fit( X_train, y_train)
prediction = dummy.predict(X_test)
print("RMSE:", root_mean_squared_error(y_test, prediction))
print("R2:", r2_score(y_test, prediction))

RMSE: 666.7611633784188
R2: -3.880265958544626e-06


### Evaluación de KNN

En esta sección, evaluaremos un modelo basado en KNN y ajustaremos los hiperparámetros más importantes, midiendo los tiempos de entrenamiento.

Inicialmente, evaluaremos el modelo con los parámetros establecidos por defecto. Esto es:
    - n_neighbors = 5
    - weights='uniform'
    - algorithm='auto'
    - leaf_size=30
    - p=2
    - metric='minkowski'
    - metric_params=None
    - n_jobs=None

In [17]:
import time
data = [[]] # Usado para almacenar los resultados
scoring_list = ["neg_root_mean_squared_error","r2"]
initial_time = time.time()

pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('knn', KNeighborsRegressor())]
)

rmse = -cross_val_score(pipe, X_train, y_train, cv=tss, scoring="neg_root_mean_squared_error").mean()
r2 = cross_val_score(pipe, X_train, y_train, cv=tss, scoring="r2").mean()
data[-1].append("KNN")
data[-1].append(rmse)
data[-1].append(r2)
data[-1].append(time.time()-initial_time)
print("RMSE:",rmse,"R2:", r2, "Time:", time.time()-initial_time)

RMSE: 449.4882966238144 R2: 0.5592102016107713 Time: 0.20499777793884277


A continuación, probamos a buscar hiperparámetros óptimos para este modelo de regresión basada en KNN. Probaremos los siguientes parámetros:
n_neighbors (probaremos con 5, 7, 9, 11, 13, 15, 17), weights (probaremos con "uniform" y "distance") y p (probaremos con 1 y 2). Para ello, utilizamos un GridSearchCV utilizando el TimeSeriesSplit creado anteriormente. 

In [18]:
initial_time = time.time()
param_grid = {"knn__n_neighbors":list(range(5,17,2)), "knn__weights":["uniform", "distance"], "knn__p":[1,2]}
pipeline_knn = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsRegressor())
])
grid_knn = GridSearchCV(pipeline_knn, param_grid, cv=tss, scoring="neg_root_mean_squared_error")
grid_knn.fit(X_train, y_train)
rmse = -grid_knn.best_score_
r2 = cross_val_score(grid_knn.best_estimator_, X_train, y_train, cv=tss, scoring="r2").mean()
data[-1].append(rmse)
data[-1].append(r2)
data[-1].append(time.time()-initial_time)
print("RMSE:",rmse,"R2:", r2, "Time:", time.time()-initial_time, "Params:", grid_knn.best_params_)

RMSE: 424.5590291743106 R2: 0.6083772037567944 Time: 1.9399971961975098 Params: {'knn__n_neighbors': 11, 'knn__p': 1, 'knn__weights': 'distance'}


### Evaluación de arboles de regresión

Ahora probaremos un modelo basado en un árbol de regresión. Inicialmente haremos la prueba con los valores predeterminados y posteriormente, en la optimización de hiperparámetros, probaremos con los siguientes hiperparámetros: criterion, max_depth, min_samples_split y min_samples_leaf.

In [19]:
from sklearn.tree import DecisionTreeRegressor

initial_time = time.time()

dtr = DecisionTreeRegressor(random_state=454455)
rmse = -cross_val_score(dtr, X_train, y_train, cv=tss, scoring="neg_root_mean_squared_error").mean()
r2 = cross_val_score(dtr, X_train, y_train, cv=tss, scoring="r2").mean()
data.append([])
data[-1].append("Arbol de regresión")
data[-1].append(rmse)
data[-1].append(r2)
data[-1].append(time.time()-initial_time)
print("RMSE:",rmse,"R2:", r2, "Time:", time.time()-initial_time)

RMSE: 533.0490891800928 R2: 0.38279628928168236 Time: 0.44299888610839844


In [20]:
initial_time = time.time()
param_grid = {"criterion":["squared_error", "absolute_error"], "max_depth":list(range(3,7,2)), "min_samples_split":list(range(41,50,2)), "min_samples_leaf":list(range(1,6,2))}

grid_tree = GridSearchCV(DecisionTreeRegressor(random_state=454455), param_grid, cv=tss, scoring="neg_root_mean_squared_error")
grid_tree.fit(X_train, y_train)
rmse = -grid_tree.best_score_
r2 = cross_val_score(grid_tree.best_estimator_, X_train, y_train, cv=tss, scoring="r2").mean()
data[-1].append(rmse)
data[-1].append(r2)
data[-1].append(time.time()-initial_time)
print("RMSE:",rmse,"R2:", r2, "Time:", time.time()-initial_time, "Params:", grid_tree.best_params_)

RMSE: 426.67567717774836 R2: 0.6032400563658257 Time: 51.09900164604187 Params: {'criterion': 'squared_error', 'max_depth': 5, 'min_samples_leaf': 3, 'min_samples_split': 45}


### Evaluación de regresión lineal

Igual que en los otros casos, primero haremos la evaluación del modelo de regresión lineal con sus hiperparámetros predeterminados y posteriormente probaremos con la optimización de éstos. Para ello, haremos el GridSearchCV con los hiperparámetros fit_intercept y positive. 

In [21]:
from sklearn.linear_model import LinearRegression

initial_time = time.time()
pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('lr', LinearRegression())]
)
rmse = -cross_val_score(pipe, X_train, y_train, cv=tss, scoring="neg_root_mean_squared_error").mean()
r2 = cross_val_score(pipe, X_train, y_train, cv=tss, scoring="r2").mean()
data.append([])
data[-1].append("Regresión lineal")
data[-1].append(rmse)
data[-1].append(r2)
data[-1].append(time.time()-initial_time)
print("RMSE:",rmse,"R2:", r2, "Time:", time.time()-initial_time)

RMSE: 573.8068591005823 R2: 0.28724890003828796 Time: 0.08899855613708496


In [22]:
initial_time = time.time()
param_grid = {"lr__fit_intercept":[True, False], "lr__positive": [True, False]}
grid_lr = GridSearchCV(pipe, param_grid, cv=tss, scoring="neg_root_mean_squared_error")
grid_lr.fit(X_train, y_train)
rmse = -grid_lr.best_score_
r2 = cross_val_score(grid_lr.best_estimator_, X_train, y_train, cv=tss, scoring="r2").mean()
data[-1].append(rmse)
data[-1].append(r2)
data[-1].append(time.time()-initial_time)
print("RMSE:",rmse,"R2:", r2, "Time:", time.time()-initial_time, "Params:", grid_lr.best_params_)

RMSE: 573.8068591005823 R2: 0.28724890003828796 Time: 0.3710010051727295 Params: {'lr__fit_intercept': True, 'lr__positive': False}


### Evaluación de regresión lineal Lasso

Ahora evaluaremos un modelo con la variación Lasso para la regresión lineal. Los hiperparámetros que optimizaremos son alph y fit_intercept. Por otro lado, para este modelo será necesario establecer un número máximo de iteraciones que hemos fijado en 10000 para evitar problemas en el notebook.

In [23]:
from sklearn.linear_model import Lasso, LassoCV

initial_time = time.time()
pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('lasso', Lasso(max_iter=10000, random_state=454455))]
)

rmse = -cross_val_score(pipe, X_train, y_train, cv=tss, scoring="neg_root_mean_squared_error").mean()
r2 = cross_val_score(pipe, X_train, y_train, cv=tss, scoring="r2").mean()
data.append([])
data[-1].append("Regresión Lineal Lasso")
data[-1].append(rmse)
data[-1].append(r2)
data[-1].append(time.time()-initial_time)
print("RMSE:",rmse,"R2:", r2, "Time:", time.time()-initial_time)

RMSE: 566.2757058941405 R2: 0.3049886546753497 Time: 0.44499850273132324


In [24]:
initial_time = time.time()
#print(list(np.logspace(-1, 0.5, 30)))

param_grid = {"lasso__alpha": list(np.logspace(-1, 0.5, 30)), "lasso__fit_intercept": [True, False]}
pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('lasso', Lasso(max_iter=15000, random_state=454455))]
)
grid_lasso = GridSearchCV(pipe, param_grid, cv=tss, scoring="neg_root_mean_squared_error")
grid_lasso.fit(X_train, np.ravel(y_train))
rmse = -grid_lasso.best_score_
r2 = cross_val_score(grid_lasso.best_estimator_, X_train, y_train, cv=tss, scoring="r2").mean()
data[-1].append(rmse)
data[-1].append(r2)
data[-1].append(time.time()-initial_time)
print("RMSE:",rmse,"R2:", r2, "Time:", time.time()-initial_time, "Params:", grid_lasso.best_params_)

RMSE: 566.2519699464789 R2: 0.30501539308289843 Time: 29.888975143432617 Params: {'lasso__alpha': 1.0826367338740546, 'lasso__fit_intercept': True}


### Evaluación de SVM

Dado que es un problema, vamos a optimizar los siguientes hiperparámetros: kernel, C, epsilon, gamma

In [25]:
from sklearn.svm import SVR

initial_time = time.time()
pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('svm', SVR())]
)

rmse = -cross_val_score(pipe, X_train, np.ravel(y_train), cv=tss, scoring="neg_root_mean_squared_error").mean()
r2 = cross_val_score(pipe, X_train, np.ravel(y_train), cv=tss, scoring="r2").mean()
data.append([])
data[-1].append("SVM")
data[-1].append(rmse)
data[-1].append(r2)
data[-1].append(time.time()-initial_time)
print("RMSE:",rmse,"R2:", r2, "Time:", time.time()-initial_time)

RMSE: 688.2117165064425 R2: -0.023585902347566152 Time: 3.1009979248046875


In [26]:
initial_time = time.time()
param_grid = {"svm__kernel":["rbf", "linear"],"svm__C": list(range(800,950,50)),"svm__epsilon": np.linspace(2,2.5,5)}
pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('svm', SVR())]
)
grid_svm = GridSearchCV(pipe, param_grid, cv=tss, scoring="neg_root_mean_squared_error")
grid_svm.fit(X_train, np.ravel(y_train))
rmse = -grid_svm.best_score_
r2 = cross_val_score(grid_svm.best_estimator_, X_train, np.ravel(y_train), cv=tss, scoring="r2").mean()
data[-1].append(rmse)
data[-1].append(r2)
data[-1].append(time.time()-initial_time)
print("RMSE:",rmse,"R2:", r2, "Time:", time.time()-initial_time, "Params:", grid_svm.best_params_)

RMSE: 397.0696148849627 R2: 0.6570609181980098 Time: 74.55903935432434 Params: {'svm__C': 900, 'svm__epsilon': 2.25, 'svm__kernel': 'rbf'}


### Resumen de resultados y conclusiones

In [27]:
headers = ["Name", "Default params RMSE", "Default params r2", "Time", "RMSE con HPO", "r2 con HPO", "Time", "Mejora (%)"]

for i in range(len(data)):
    for j in range(len(data[i])):
        if j >0:
            data[i][j] = round(data[i][j], 3)
    data[i].append(round((data[i][1] / data[i][4]-1)*100, 3))

df = pd.DataFrame(data= data, columns=headers).style.hide()
df

Name,Default params RMSE,Default params r2,Time,RMSE con HPO,r2 con HPO,Time,Mejora (%)
KNN,449.488000,0.559000,0.205000,424.559000,0.608000,1.940000,5.872000
Arbol de regresión,533.049000,0.383000,0.443000,426.676000,0.603000,51.099000,24.931000
Regresión lineal,573.807000,0.287000,0.089000,573.807000,0.287000,0.371000,0.000000
Regresión Lineal Lasso,566.276000,0.305000,0.445000,566.252000,0.305000,29.889000,0.004000
SVM,688.212000,-0.024000,3.101000,397.070000,0.657000,74.559000,73.323000


En la tabla podemos visualizar el RMSE y el coeficiente de determinación R² para diferentes modelos. En la tabla podemos ver el RMSE con los parámetros por defecto para cada uno de los modelos, así como el RMSE una vez hecha la optimización de hiperparámetros. Como se ve, los modelos que mejores resultados presentan son SVM, KNN y los árboles de decisión, que tienen los valores para RMSE más bajos y para R² más altos. La regresión lineal es, por otro lado, el método que peores resultados ofrece.
Se puede concluir que el modelo que mejor resultado arroja es SVM con los parámetros {'svm__C': 900, 'svm__epsilon': 2.25, 'svm__kernel': 'rbf'}. Serán estos los valores que utilizaremos en el modelo final, dado que presenta el coeficiente de determinación más alto y el RMSE más bajo. 

Por otro lado, si vemos los casos con los parámetros por omisión, podemos determinar que los métodos más rápidos son KNN y la regresión lineal, mientras que métodos como SVM son más lentos. Por otra parte, se ve que para encontrar los hiperparámetros óptimos, también es más lento SVM (SVR) que el resto, mientras que la optimización de hiperparámetros es muy rápida con la regresión lineal. También podemos ver que comparativamente, es bastante más lenta la optimización de hiperparámetros si utilizamos la regresión lineal Lasso en vez de la normal.

Salvo en el caso de SVM con los parámetros por defecto, en todos los casos el regresor dummy ofrece un resultado peor, por lo que los modelos son una mejor opción que usar este regresor. Por otro lado, podemos ver que todos los modelos mejoran su actuación una vez sus hiperparámetros han sido optimizados, salvo en la regresión lineal, que sus mejores hiperparámetros son los que se usan por defecto.
Como se ve en la última columna, el modelo que mejora más con la optimización de los hiperparámetros es SVM, seguido del árbol de regresión. 

Respecto a los atributos relevantes, si observamos los coeficientes del estimador de la Regresión Lineal Lasso, aquellos que tengan valor 0, se considera que no aportan suficiente información para ser relevantes. A continuación, podemos ver el coeficiente de cada atributo y los atributos que tienen coeficiente nulo.

In [28]:
coeficients = grid_lasso.best_estimator_["lasso"].coef_
attributes = wind_ava.drop(['energy'], axis='columns').columns
df = pd.DataFrame({"Atributos":attributes, "Coeficientes Lasso": coeficients})
df

,Atributos,Coeficientes Lasso
0,p54.162.13,7.216149
1,p55.162.13,17.763774
2,cape.13,-26.419103
3,p59.162.13,244.109584
4,lai_lv.13,11.082537
5,lai_hv.13,0.000000
6,u10n.13,-525.715303
7,v10n.13,0.000000
8,sp.13,-56.526289
9,stl1.13,-0.000000


In [29]:
df[df["Coeficientes Lasso"]==0]

,Atributos,Coeficientes Lasso
5,lai_hv.13,0.0
7,v10n.13,0.0
9,stl1.13,-0.0
14,stl3.13,-0.0


Estos 4 atributos no los consideramos relevantes, es decir no aportan información sobre la variable independiente que es la energía. Estos atributos representan:
* lai_hv: Leaf area index, high vegetation
* v10n: Neutral wind at 10 m v-component
* stl1: Soil temperature level 1
* stl3: Soil temperature level 3

## Evaluación del rendimiento y  entrenamiento y uso del modelo final

Para el modelo final, en primer lugar haremos una estimación del comportamiento de nuestro modelo. Para eso, utilizaremos los datos del conjunto de train para entrenar y los datos del conjunto de test para evaluar. Lo haremos con el modelo que ofreció los mejores resultados en la fase anterior, es decir, SVR con "rfb" como kernel y unos valores de C=900 y epsilon=2.25.

Sin embargo, para hacer el entrenamiento final, utilizaremos todos los datos disponibles. Es por esto que la estimación que hacemos es pesimista, dado que luego entrenamos con más datos en realidad. 

In [30]:
from joblib import dump, load
from sklearn import metrics

# Hacemos cross validation con todos los datos para determinar el desempeño futuro de nuestro modelo.
pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('svm', SVR(kernel="rbf", C=900, epsilon=2.25))]
)
X = wind_ava.drop(['energy'], axis='columns')
y = wind_ava[['energy']]

pipe.fit(X_train, np.ravel(y_train))
predictions = pipe.predict(X_test)
rmse = root_mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f"RMSE: {rmse}")
print(f"r2: {r2}")

# Guardamos el modelo final
final_pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('svm', SVR(kernel="rbf", C=900, epsilon=2.25))]
)

final_pipe.fit(X, np.ravel(y))
dump(final_pipe, 'modelo_final.pkl') 

RMSE: 386.7961147673092
r2: 0.6634688264358362


['modelo_final.pkl']

Una vez que hemos exportado nuestro modelo, podemos importarlo y utilizarlo para hacer predicciones en cualquier otro programa. Por ejemplo, podemos utilizar el modelo final para hacer las predicciones para la competición.

In [31]:
# Ahora podemos cargar nuestro modelo y hacer las predicciones para la competición
fm = load('modelo_final.pkl')

wind_comp = pd.read_csv('dataset/wind_comp.csv.gz', compression="gzip")
wind_comp["datetime"] = pd.to_datetime(wind_comp["datetime"])
wind_comp.set_index("datetime", inplace=True)
for c in wind_comp.columns:
  if not c.endswith(".13") and c != "energy":
    wind_comp.drop(c, axis=1, inplace=True)

predict = fm.predict(wind_comp) 
wind_comp.insert(0,"predictions", predict)
wind_comp["predictions"].to_csv('predicciones.csv')

## Uso de ChatGPT

Para la realización de esta práctica no hemos utilizado la herramienta de IA generativa ChatGPT. Toda la información y ejemplos de códiog ha sido obtenida del temario de la asignatura, la documentación de las diferentes librerias utilizadas y foros de dudas ampliamente conocidos como StackOverflow. En especial, nos gustaría remarcar la documentación de scikit-learn, librería en la que se basa el grueso de la práctica, que contiene de manera detallada el funcionamiento de los diferentes parámetros de los clasificadores y regresores, así como pequeños ejemplos de código que facilitan su uso. Foros como StackOverflow han sido especialmente útiles para trabajar con librerías cuyo uso puede ser complejo pero a su vez están muy extendidas como pandas o numpy.